In [1]:
import pickle
import lzma
from collections import OrderedDict

import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [2]:
from exfor_tools.distribution import AngularDistribution

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
from jitr.optical_potentials import kduq, wlh

In [4]:
import rxmc

In [5]:
import elm

In [6]:
neutron = (1, 0)
proton = (1, 1)

In [7]:
kduq_params_nn = kduq.get_samples_federal(neutron)
kduq_params_pp = kduq.get_samples_federal(proton)

In [8]:
nparams_nn = len(kduq_params_nn[0])
nparams_pp = len(kduq_params_pp[0])

## read in measurements and pre-computed workspaces

In [9]:
from pathlib import Path

evidence_dir = Path("../evidence/")
output_dir = Path("./kduq_propagation//")
output_dir.mkdir(parents=True, exist_ok=True)

In [10]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

## set up models

In [13]:
nn_model = rxmc.elastic_diffxs_model.ElasticDifferentialXSModel(
    "dXS/dA",
    kduq.central,
    kduq.spin_orbit,
    lambda x, ws: kduq.calculate_params(
        ws.rxn.projectile, ws.rxn.target, ws.kinematics.Elab, *x
    ),
    "KDUQ (n,n)",
)
pp_model = rxmc.elastic_diffxs_model.ElasticDifferentialXSModel(
    "dXS/dRuth",
    kduq.central_plus_coulomb,
    kduq.spin_orbit,
    lambda x, ws: kduq.calculate_params(
        ws.rxn.projectile, ws.rxn.target, ws.kinematics.Elab, *x
    ),
    "KDUQ (p,p)",
)

## Propagate KDUQ into constraint observables

In [14]:
yth_nn = []
for i, constraint in enumerate(tqdm(kduq_nn_corpus.constraints)):
    yth_nn.append(np.vstack([constraint.model(p) for p in kduq_params_nn]))

  0%|                                                                        | 0/65 [00:00<?, ?it/s]/home/beyerk/Projects/jitr/src/jitr/optical_potentials/kduq.py:384: RuntimeWarning: overflow encountered in exp
  1 + np.exp((A - params["d2_A3"]) / params["d2_A2"])
100%|███████████████████████████████████████████████████████████████| 65/65 [01:00<00:00,  1.07it/s]


In [15]:
yth_pp = []
for i, constraint in enumerate(tqdm(kduq_pp_corpus.constraints)):
    yth_pp.append(np.vstack([constraint.model(p) for p in kduq_params_pp]))

  0%|                                                                       | 0/160 [00:00<?, ?it/s]/home/beyerk/Projects/jitr/src/jitr/optical_potentials/kduq.py:384: RuntimeWarning: overflow encountered in exp
  1 + np.exp((A - params["d2_A3"]) / params["d2_A2"])
100%|█████████████████████████████████████████████████████████████| 160/160 [02:16<00:00,  1.17it/s]


## Write posterior predictive distributions to disk

In [16]:
with open(output_dir / "nn_kduq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [17]:
with open(output_dir / "pp_kduq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)